In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=True)

In [8]:
from controllers.directional_trading.macd_bb_rsi_multi_token_controller import MACDBBRSIMultiTokenControllerConfig
import datetime
from decimal import Decimal

# Controller configuration
exchange = "binance_perpetual"
trading_pairs = ["WLD-USDT"]
candles_exchange = "binance_perpetual"
candles_pairs = ["WLD-USDT"]
candles_interval = "5m"
macd_fast = 6
macd_slow = 13
macd_signal = 4
bb_length = 20
bb_std = 2.2
bb_long_threshold = 0.2
bb_short_threshold = 0.8
rsi_low = 20
rsi_high = 80
rsi_length = 14
order_amount_quote = 100
take_profit = 0.015
stop_loss = 0.01
time_limit = 60 * 60 * 1

start = int(datetime.datetime(2025, 7, 1).timestamp())
end = int(datetime.datetime(2025, 7, 9).timestamp())

# Creating the instance of the configuration and the controller
config = MACDBBRSIMultiTokenControllerConfig(
    exchange=exchange,
    trading_pairs=trading_pairs,
    candles_exchange=candles_exchange,
    candles_pairs=candles_pairs,
    candles_interval=candles_interval,
    macd_fast=macd_fast,
    macd_slow=macd_slow,
    macd_signal=macd_signal,
    bb_length=bb_length,
    bb_std=bb_std,
    bb_long_threshold=bb_long_threshold,
    bb_short_threshold=bb_short_threshold,
    rsi_low=rsi_low,
    rsi_high=rsi_high,
    rsi_length=rsi_length,
    order_amount_quote=Decimal(order_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    time_limit=time_limit,
)

In [9]:
backtesting_result = await backtesting.run_backtesting(config, start, end, candles_interval)

id=None controller_name='macd_bb_rsi_multi_token' controller_type='generic' total_amount_quote=<property object at 0x32bda6160> manual_kill_switch=False candles_config=[] initial_positions=[] exchange='binance_perpetual' trading_pairs=['WLD-USDT'] candles_exchange='binance_perpetual' candles_pairs=['WLD-USDT'] candles_interval='5m' candles_length=60 rsi_low=20.0 rsi_high=80.0 rsi_length=14 macd_fast=6 macd_slow=13 macd_signal=4 bb_length=20 bb_std=2.2 bb_long_threshold=0.2 bb_short_threshold=0.8 order_amount_quote=Decimal('100') leverage=10 position_mode='ONEWAY' stop_loss=Decimal('0.01000000000000000020816681711721685132943093776702880859375') take_profit=Decimal('0.01499999999999999944488848768742172978818416595458984375') time_limit=3600


In [10]:
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()

AttributeError: 'int' object has no attribute 'get'

In [ ]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
import pandas as pd

executors_df = backtesting_result.executors_df
executors_df.head()

### Backtesting Analysis

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [ ]:
import plotly.express as px

# Create a new column for profitability
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

# Create the scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y='net_pnl_quote',
    title='PNL per Trade',
    color='profitable',
    color_discrete_map={True: 'green', False: 'red'},
    labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PNL (Quote)'},
    hover_data=['filled_amount_quote', 'side']
)

# Customize the layout
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',  # Dark background
    paper_bgcolor='rgba(0,0,0,0.8)',  # Dark background for the entire plot area
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

# Add a horizontal line at y=0 to clearly separate profits and losses
fig.add_hline(y=0, line_dash="dash", line_color="lightgray")

# Show the plot
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [ ]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()


# Conclusion
We can see that the indicator has potential to bring good signals to trade and might be interesting to see how we can design a market maker that shifts the mid price based on this indicator.
A lot of the short signals are wrong but if we zoom in into the loss signals we can see that the losses are not that big and the wins are bigger and if we had implemented the trailing stop feature probably a lot of them are going to be profits.

# Next steps
- Filter only the loss signals and understand what you can do to prevent them
- Try different configuration values for the indicator
- Test in multiple markets, pick mature markets like BTC-USDT or ETH-USDT and also volatile markets like DOGE-USDT or SHIB-USDT